# Clase 9

- Fecha: 27-10-2022
- Unidad 2: ecosistema científico básico
    - Pandas continuación
    - Plotly continuación

## Pandas 

- Operaciones de agregación: ``sort()``, ``groupby()``, ``sum()``, etc 

In [ ]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go

import plotly.express as px       # atencion, plotly EXPRESS! 

In [ ]:
gps = pd.read_csv("datos/gps_sin_col_name.CSV", sep=";", names=["lidar", "timestamp", "lat", "lon", "altgps", "velocidad", "altbar", "patm"], parse_dates=["timestamp"])
gps.head()

In [ ]:
gps.info()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scattergl(x=gps["timestamp"], y=gps["altgps"]))
fig.show()

### Eliminar columnas y filas

- ¿Elimina en el dataframe original?

Con el método ``.drop()`` obtenemos un nuevo dataframe sin las columnas eliminadas.

In [ ]:
gps_tll = gps.drop(columns=["lidar", "altgps", "velocidad", "altbar", "patm"])

In [ ]:
gps_tll.head()

In [ ]:
gps

### Ordenamiento

- ¿Modifica el dataframe original?

In [ ]:
gspatm = gps.sort_values(by="patm")
gspatm

### Boolean Indexing
- Igual que numpy (porque en el fondo es numpy!!)

In [ ]:
gps["velocidad"]>5

In [ ]:
ind = gps["velocidad"]>7
gps[ind]

### Ejercicio 9.1

- Utilizando el dataset del archivo ``datos/importaciones.csv`` 
    - ¿qué productos no se importaron en el año 2015?
    - ¿en qué productos la importación **fue mayor al doble** entre el 2015 y 2016?

In [ ]:
imp = pd.read_csv("datos/importaciones.csv")
imp.tail()

In [ ]:
no_imp_2015 = imp["Enero al 10 de Diciembre 2015"]==0
imp[no_imp_2015]

In [ ]:
ind = imp["Enero al 10 de Diciembre 2016"] > (2*imp["Enero al 10 de Diciembre 2015"])
imp[ind]

### Agrupamiento
- El método ``group by`` involucra:
    - Dividir los datos según algún criterio de selección
    - Aplicar alguna función a esos grupos en forma independiente
    - Combinar esos resultados en una estructura independiente
    
Veamos nuevamente los datos de eph

In [ ]:
eph = pd.read_csv("datos/eph_pobreza2019.csv", sep=";")
eph.head()

- **Agrupamos por regiones**

In [ ]:
grupo_reg = eph.groupby("Regiones")

- **¿Qué es ``grupo_reg``?**

In [ ]:
type(grupo_reg)

- **Veamos algunos de sus métodos**

In [ ]:
grupo_reg.groups

- **¿Será iterable?**

In [ ]:
for nombre, grupo in grupo_reg:
    print("====================")
    print(nombre)
    print("====================")
    print(grupo)
    print()

- **Seleccionemos algún grupo**

In [ ]:
grupo_reg.max()

- Podemos aplicar operaciones a cada uno de esos grupos **(cada grupo es un DataFrame!!!)**. Recordemos el resultado de aplicar ``.sum()`` al DataFrame completo

In [ ]:
eph.sum()

### Guardar datos en archivos CSV o binarios



- Para almacenar los datos de un dataframe en un archivo csv basta con ejecutar: ``df.to_csv("nomarchi.csv")``


- Para almacenar directamente los dataframes:
    - Guardar: ``df.to_pickle("mi_dataframe.p")``
    - Leer: ``df = pd.read_pickle("mi_dataframe.p")``
    
En python estandar tenemos el módulo ``pickle`` que nos permite guardar objetos (listas, diccionarios, etc) en archivos de disco, en pandas tenemos el método ``to_pickle()`` para hacer lo mismo con objetos dataframes.

In [ ]:
# con esto lo guardo al dataframe en un archivo .csv
gps_tll.to_csv("pruebitagps.csv", index=False)

In [ ]:
eph_hogares = eph.drop(columns=["Regiones", "PobrezaPersonas", "IndigenciaPersonas"])

# con esto lo guardo en formato BINARIO!
eph_hogares.to_pickle("mi_eph_hogares.p")

In [ ]:
# con esto lo leo a ese archivo BINARIO

eph_leido = pd.read_pickle("mi_eph_hogares.p")
eph_hogares.head()

### Ejercitación 9.2
- Muestre el dataframe donde la pobreza de hogares ha superado la media (ayuda: boolean indexing)
- Ordene el resultado previo por alguna de las columnas
- Obtener la suma de los índices de pobreza e indigencia agrupados por Regiones
- Muestre los resultados ordenandolos por la columna de pobreza hogares
- **Usando el archivo ``importaciones.csv`` obtenga la suma de las columnas agrupadas por Rubro**

## Visualización con Plotly (2da parte)

- https://plot.ly/python/
- https://www.plotly.express/plotly_express/

Existen dos modos de uso:

- Usando el submódulo *express*: facilita generar gráficos sencillos, para exploración rápida **de DataFrames**

```python
import plotly.express as px
px.line(...)

```

- Usando *plotly graph objects*

```python
import plotly.graph_objects as go

# creamos una figura
fig = go.Figure()
# agregamos la traza
fig.add_trace(go.Scatter(x=x, y=y,mode='markers', name='Observaciones'))
# lo mostramos
fig.show()
```

### Templates

Tenemos un conjunto de templates predefinidos para los gráficos: 
    - plotly
    - plotly_white
    - plotly_dark
    - ggplot2
    - seaborn
    - none

In [ ]:
# cargamos el dataset
altura = pd.read_csv("datos/barometro.CSV", sep=";")
altura.head()

In [ ]:
fig = px.scatter(altura, y="altimetro", title="con plotly express")
fig.show()

In [ ]:
for un_template in ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "none"]:
    fig = px.scatter(altura, y="altimetro", template=un_template, title=f"Template: {un_template}")
    fig.show()

### Gráficos estadísticos

- https://plot.ly/python/box-plots/

In [ ]:
fig = go.Figure()

fig.add_trace(go.Box(y=eph["PobrezaHogares"],     name="PobrezaHogares"))
fig.add_trace(go.Box(y=eph["IndigenciaHogares"],  name="IHogares"))
fig.add_trace(go.Box(y=eph["PobrezaPersonas"],    name="PobrezaP"))
fig.add_trace(go.Box(y=eph["IndigenciaPersonas"], name="IPersonas"))

fig.show()

### Datos sobre Mapas

- https://plot.ly/python/reference/#layout-geo
- Leemos el archivo ``PBI_mundiales.csv``

In [ ]:
pbi = pd.read_csv("datos/PBI_mundiales.csv", sep=",")

#pbi = pbi.dropna()

In [ ]:
fig = px.scatter_geo(pbi, locations="iso_alpha", color="continent",
                     hover_name="country", size="pop",
                     projection="natural earth")
fig.show()

- Agreguemos el campo ``animation_frame="year"``

In [ ]:
fig = px.scatter_geo(pbi, locations="iso_alpha", color="continent",
                     hover_name="country", size="pop",
                     animation_frame="year",
                     projection="natural earth")
fig.show()

### Mapas online
- Ver mapbox: https://www.mapbox.com/

In [ ]:
caminata = pd.read_csv("datos/gps.CSV", sep=";")
caminata.info()

In [ ]:
fig = px.scatter_mapbox(caminata, lat="lat", lon="lng", hover_name="altgps", 
                        hover_data=["timegps", "pAtm", "altbar"],
                        color_discrete_sequence=["fuchsia"],
                        zoom=13, height=500)
fig.update_layout(mapbox_style="open-street-map")
fig.show()

**Vamos a animarlo!!**

In [ ]:
caminata = pd.read_csv("datos/gps.CSV", sep=";")
caminata.head()

fig = px.scatter_mapbox(caminata, lat="lat", lon="lng", hover_name="altgps", hover_data=["timegps", "pAtm", "altbar"],
                        color_discrete_sequence=["fuchsia"],
                        animation_frame="timegps",
                        zoom=13, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### Visualización 3d

In [ ]:
iris = pd.read_csv("datos/iris_data.csv")
iris

In [ ]:
fig = px.scatter_3d(iris, x='sepal.length', y='sepal.width', z='petal.width', color='variety')
fig.show()

In [ ]:
canal = pd.read_csv("datos/canal.csv")

canal.tail()

In [ ]:
figure = go.Figure()

figure.add_trace(
    go.Scatter3d(x = canal.x, y = canal.y, z = -canal.z, mode = 'markers', marker = {"size":1.5})
)
figure.update_layout(title='Nube de puntos', width=800, height=600,)
figure.show()

In [ ]:
import plotly.graph_objects as go
import pandas as pd

z_data = pd.read_csv('datos/terreno.csv')

fig = go.Figure(data=[go.Surface(z=z_data.values)])

fig.update_layout(title='Elevación de terreno', autosize=False,
                  width=800, height=600,
                  margin=dict(l=65, r=50, b=65, t=90))

fig.show()

In [ ]:
lidar1 = pd.read_csv('datos/conventosanfrancisco1.csv.xyz')
lidar1

In [ ]:
traza = go.Scatter3d(x = lidar1.x, y = lidar1.y, z = lidar1.z, mode = 'markers', marker = dict(size=1))

figure = go.Figure()
figure.add_trace(traza)

figure.update_layout(title='Convento San Francisco', height=800,)

figure.show()

In [ ]:
lidar2 = pd.read_csv('datos/museo1.csv.xyz')

traza = go.Scatter3d(x = lidar2.x, y = lidar2.y, z = lidar2.z, mode = 'markers', marker = dict(size=1.5))

figure = go.Figure()
figure.add_trace(traza)

figure.update_layout(title='Museo etnografico', height=800,)

figure.show()

In [ ]:
futon = pd.read_csv('datos/futon1.csv.xyz')

traza = go.Scatter3d(x = futon.x, y = futon.y, z = futon.z, mode = 'markers', marker = dict(size=0.9))

figure = go.Figure()
figure.add_trace(traza)

figure.update_layout(title='Futón', height=800,)

figure.show()